In [91]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
#Importing libraries

import nltk, re, string
from nltk.corpus import stopwords, twitter_samples
import pandas as pd
import numpy as np
import csv
import pickle

In [93]:
#Reading data

df_train = pd.read_csv('/content/drive/MyDrive/tweets_hate_speech/train_E6oV3lV.csv',quoting=3, error_bad_lines=False).set_index('id')

df_train.head(20)

b'Skipping line 21: expected 3 fields, saw 4\nSkipping line 34: expected 3 fields, saw 4\nSkipping line 40: expected 3 fields, saw 4\nSkipping line 43: expected 3 fields, saw 5\nSkipping line 56: expected 3 fields, saw 4\nSkipping line 59: expected 3 fields, saw 4\nSkipping line 75: expected 3 fields, saw 5\nSkipping line 77: expected 3 fields, saw 5\nSkipping line 79: expected 3 fields, saw 4\nSkipping line 89: expected 3 fields, saw 4\nSkipping line 95: expected 3 fields, saw 4\nSkipping line 97: expected 3 fields, saw 4\nSkipping line 100: expected 3 fields, saw 4\nSkipping line 105: expected 3 fields, saw 4\nSkipping line 114: expected 3 fields, saw 4\nSkipping line 122: expected 3 fields, saw 4\nSkipping line 123: expected 3 fields, saw 4\nSkipping line 141: expected 3 fields, saw 5\nSkipping line 145: expected 3 fields, saw 4\nSkipping line 157: expected 3 fields, saw 4\nSkipping line 158: expected 3 fields, saw 4\nSkipping line 160: expected 3 fields, saw 4\nSkipping line 169: e

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation
6,0,[2/2] huge fan fare and big talking before the...
7,0,@user camping tomorrow @user @user @user @use...
8,0,the next school year is the year for exams.ð...
9,0,we won!!! love the land!!! #allin #cavs #champ...


In [94]:
!pip install emoji --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [96]:
#Cleaning the tweets

import re
from nltk import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from unicodedata import normalize

def cleaner(tweet):
    tweet = "".join(u for u in tweet if u not in ("?", ".", ",", ";", ":", "!", '"', "'", "¡", "¿"))
    tweet = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", tweet), 0, re.I)
    tweet = normalize( 'NFC', tweet) #Esta linea y la anterior transforma á -> a, é -> e ....
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split()) #Removes the @user
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r"[^a-zA-Z]", " ", str(tweet).lower())
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    
    # Tokenize text
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    token = tokenizer.tokenize(tweet)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in token if t not in stop]
    
    # Lemmatization
    lem = " ".join(temp for temp in [WordNetLemmatizer().lemmatize(w) for w in words])
    
    return lem

df_train['tweet'] = df_train['tweet'].map(lambda x: cleaner(x))
df_train['tweet'].head(20)

id
1     father dysfunctional selfish drag kid dysfunct...
2     thanks lyft credit cant use cause dont offer w...
3                                        bihday majesty
4                           model love u take u time ur
5                         factsguide society motivation
6     huge fan fare big talking leave chaos pay disp...
7                               camping tomorrow dannya
8     next school year year exam cant think school e...
9     love land allin cavs champion cleveland clevel...
10                                        welcome im gr
11    ireland consumer price index mom climbed previ...
12    selfish orlando standwithorlando pulseshooting...
13                   get see daddy today day gettingfed
14    cnn call michigan middle school build wall cha...
15    comment australia opkillingbay seashepherd hel...
16              ouchjunior angry got junior yugyoem omg
17                     thankful paner thankful positive
18                                        ret

In [97]:
label_0 = len(df_train[df_train['label']==0]) 
label_1 = len(df_train[df_train['label']==1])
perc_0 = label_0/(label_0+label_1)*100
perc_1 = label_1/(label_0+label_1)*100

print(f'There is {label_1} hate speech tweets, which represents {perc_1:.2f}%')
print(f'There is {label_0} non hate speech tweets, which represents {perc_0:.2f}%')

There is 594 hate speech tweets, which represents 6.44%
There is 8636 non hate speech tweets, which represents 93.56%


##Random oversampling

In [ ]:
count_non_hate = df_train[df_train['label'] == 0]['tweet'].count()
df_hate_speech = df_train[df_train['label'] == 1]
df_non_hate_speech = df_train[df_train['label'] == 0]
df_hate_speech_oversample = df_hate_speech.sample(count_non_hate, replace=True)
train_df_oversampled = pd.concat([df_non_hate_speech, df_hate_speech_oversample], axis=0)
df_train = train_df_oversampled

print('Random over-sampling:')
print(train_df_oversampled['label'].value_counts())

Random over-sampling:
0    8636
1    8636
Name: label, dtype: int64


##Word embedding (GloVe)

In [ ]:
#Download GloVe
!wget http://nlp.stanford.edu/data/glove.6B.zip
#Unzip
!unzip glove*.zip

--2022-08-23 16:56:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-08-23 16:56:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-08-23 16:56:07--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
#Getting the path for GloVe files
!ls
!pwd

drive		   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
/content


In [ ]:
#Index the vectors 
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [ ]:
#Fuse with google drive
!pip install --upgrade pip
!pip install -U -q pydrive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=None&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=None&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Cannot retrieve auth tokens.
Failure("Unexpected error response: {\n  \"error\": \"invalid_client\",\n  \"error_description\": \"The OAuth client was not found.\"\n}")
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/x

In [ ]:
#Run this cell to save indexed vectors for re-use
import pickle
pickle.dump({'embeddings_index' : embeddings_index } , open('drive/path/to/your/file/location', 'wb'))

In [98]:
#Once downloaded to google colab we can use it directly

#Index the vectors 
print('Indexing word vectors.')

embeddings_index = {}
f = open('/content/drive/MyDrive/Glove_word_embeddings/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 387930 word vectors.


In [99]:
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 387930 word vectors.


##Tokenizer

In [100]:
# add one column for encoding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings('ignore')

#List of tweets
tweets = list(df_train['tweet'])
likes = list(df_train['label'])

# tokenize tweet text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweets)
encoded_docs = tokenizer.texts_to_sequences(tweets)

# convert string to int array
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

# set encode values for each tweet
tweets = []
for pad in padded_sequence:
    tweets.append(list(pad))

## Print tweets 0 and 1 tokenized
print(tweets[0])
print(tweets[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 2248, 2249, 135, 379]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [101]:
from numpy import zeros

vocab_size = len(tokenizer.word_index) + 1
print(len(tokenizer.word_index))
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

16522


##Model Building

In [102]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding, Masking, LSTM, Dropout

# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=200, trainable=False)
model.add(e)
model.add(Masking(mask_value=0.0))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 100)          1652300   
                                                                 
 masking_2 (Masking)         (None, 200, 100)          0         
                                                                 
 dense_4 (Dense)             (None, 200, 64)           6464      
                                                                 
 dropout_2 (Dropout)         (None, 200, 64)           0         
                                                                 
 flatten_2 (Flatten)         (None, 12800)             0         
                                                                 
 dense_5 (Dense)             (None, 2)                 25602     
                                                                 
Total params: 1,684,366
Trainable params: 32,066
Non-t

In [146]:
from tensorflow.keras.utils import to_categorical
# fit the model
tweets = np.array(tweets)
likes = np.array(likes)
print(likes[0])
likes_cat = to_categorical(likes, 2)
print(likes_cat[0])
model.fit(tweets, likes_cat, epochs=20, verbose=1)

0
[1. 0.]
Epoch 1/20
289/289 [==============================] - 6s 22ms/step - loss: 0.0499 - accuracy: 0.9813
Epoch 2/20
289/289 [==============================] - 6s 21ms/step - loss: 0.0460 - accuracy: 0.9827
Epoch 3/20
289/289 [==============================] - 6s 20ms/step - loss: 0.0401 - accuracy: 0.9852
Epoch 4/20
289/289 [==============================] - 6s 21ms/step - loss: 0.0459 - accuracy: 0.9827
Epoch 5/20
289/289 [==============================] - 6s 20ms/step - loss: 0.0465 - accuracy: 0.9826
Epoch 6/20
289/289 [==============================] - 6s 20ms/step - loss: 0.0446 - accuracy: 0.9837
Epoch 7/20
289/289 [==============================] - 6s 21ms/step - loss: 0.0414 - accuracy: 0.9852
Epoch 8/20
289/289 [==============================] - 6s 20ms/step - loss: 0.0448 - accuracy: 0.9833
Epoch 9/20
289/289 [==============================] - 6s 21ms/step - loss: 0.0425 - accuracy: 0.9841
Epoch 10/20
289/289 [==============================] - 6s 21ms/step - loss: 0.041

In [147]:
loss, accuracy = model.evaluate(tweets, likes_cat, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 99.458289


In [148]:
predict = model.predict(tweets)
predict

array([[9.9858558e-01, 1.4144732e-03],
       [1.0000000e+00, 3.3387865e-13],
       [9.9739093e-01, 2.6091300e-03],
       ...,
       [9.9996352e-01, 3.6521284e-05],
       [9.9998474e-01, 1.5251761e-05],
       [9.9999964e-01, 3.9581781e-07]], dtype=float32)

In [149]:
tweets

array([[   0,    0,    0, ..., 2249,  135,  379],
       [   0,    0,    0, ..., 3657, 2749, 3658],
       [   0,    0,    0, ...,    0,   15, 1887],
       ...,
       [   0,    0,    0, ...,  244, 1497,  222],
       [   0,    0,    0, ...,   20,   45, 2319],
       [   0,    0,    0, ...,  240,   78,  211]], dtype=int32)

In [164]:
##Insert tweet to predict if it is hate or non-hate speech

test_tweet= ["Great job bro!"]

#Cleaning
test_tweet = [cleaner(test_tweet)]

#Tokenize
encoded_tweet = tokenizer.texts_to_sequences(test_tweet)

# convert string to int array
padded_tweet = pad_sequences(encoded_tweet, maxlen=200)

predict = model.predict(padded_tweet)
print(predict)

second_index= predict[0][1]
if (second_index> 0.15):
  print("Hate speech tweet")
else:
  print("Non-hate speech")


[[0.92222697 0.07777306]]
Non-hate speech


##Performance with test data

In [166]:
#Reading data

df_test = pd.read_csv('/content/drive/MyDrive/tweets_hate_speech/test_tweets_anuFYb8.csv',quoting=3, error_bad_lines=False).set_index('id')

df_test.head(20)

b'Skipping line 5: expected 2 fields, saw 4\nSkipping line 6: expected 2 fields, saw 3\nSkipping line 15: expected 2 fields, saw 4\nSkipping line 17: expected 2 fields, saw 3\nSkipping line 20: expected 2 fields, saw 3\nSkipping line 31: expected 2 fields, saw 3\nSkipping line 39: expected 2 fields, saw 4\nSkipping line 40: expected 2 fields, saw 3\nSkipping line 45: expected 2 fields, saw 4\nSkipping line 66: expected 2 fields, saw 3\nSkipping line 67: expected 2 fields, saw 3\nSkipping line 72: expected 2 fields, saw 3\nSkipping line 74: expected 2 fields, saw 3\nSkipping line 75: expected 2 fields, saw 3\nSkipping line 78: expected 2 fields, saw 3\nSkipping line 82: expected 2 fields, saw 3\nSkipping line 96: expected 2 fields, saw 3\nSkipping line 99: expected 2 fields, saw 3\nSkipping line 104: expected 2 fields, saw 3\nSkipping line 124: expected 2 fields, saw 4\nSkipping line 139: expected 2 fields, saw 3\nSkipping line 148: expected 2 fields, saw 3\nSkipping line 152: expected 

,tweet
id,
31963,#studiolife #aislife #requires #passion #dedic...
31964,@user #white #supremacists want everyone to s...
31965,safe ways to heal your #acne!! #altwaystohe...
31968,choose to be :) #momtips
31969,something inside me dies ð¦ð¿â¨ eyes nes...
31970,#finished#tattoo#inked#ink#loveitâ¤ï¸ #â¤ï¸...
31971,@user @user @user i will never understand why...
31972,#delicious #food #lovelife #capetown mannaep...
31973,1000dayswasted - narcosis infinite ep.. make m...


In [172]:
def pipeline_tweets(test_tweet):
  test_tweet=[test_tweet]
  #Cleaning
  test_tweet = [cleaner(test_tweet)]

  #Tokenize
  encoded_tweet = tokenizer.texts_to_sequences(test_tweet)

  # convert string to int array
  padded_tweet = pad_sequences(encoded_tweet, maxlen=200)

  return padded_tweet

df_test['predicted']=df_test['tweet'].apply(pipeline_tweets)
df_test['predicted']

id
31963    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
31964    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
31965    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
31968    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
31969    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
                               ...                        
49154    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
49155    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
49156    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
49157    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
49159    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
Name: predicted, Length: 14695, dtype: object

In [182]:
def hate_trigger(prediction):
  second_index= prediction[0][1]
  if (second_index> 0.15):
    return 1
  else:
    return 0

df_test['predicted']=df_test['predicted'].apply(model.predict)
df_test['predicted']

id
31963     [[0.99961734, 0.0003826593]]
31964       [[0.013937243, 0.9860627]]
31965    [[0.99999976, 2.3223174e-07]]
31968        [[0.9832298, 0.01677015]]
31969    [[0.99998784, 1.2214697e-05]]
                     ...              
49154     [[0.9999999, 1.1949092e-07]]
49155      [[0.0006213037, 0.9993787]]
49156            [[1.0, 8.890959e-09]]
49157        [[0.90022314, 0.0997768]]
49159      [[0.9999957, 4.240461e-06]]
Name: predicted, Length: 14695, dtype: object

In [184]:
df_test['predicted']=df_test['predicted'].apply(hate_trigger)
df_test.head(20)

,tweet,predicted
id,,
31963,#studiolife #aislife #requires #passion #dedic...,0
31964,@user #white #supremacists want everyone to s...,1
31965,safe ways to heal your #acne!! #altwaystohe...,0
31968,choose to be :) #momtips,0
31969,something inside me dies ð¦ð¿â¨ eyes nes...,0
31970,#finished#tattoo#inked#ink#loveitâ¤ï¸ #â¤ï¸...,0
31971,@user @user @user i will never understand why...,0
31972,#delicious #food #lovelife #capetown mannaep...,0
31973,1000dayswasted - narcosis infinite ep.. make m...,0
